<a href="https://colab.research.google.com/github/Robin39-AFS/Author-s-Gender-Prediction-Based-on-Their-Bangla-Text/blob/main/OverSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import csv
import sys
import pandas as pd
import numpy as np
from re import sub
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/dataset.csv')
df.head()

,author,gender,title,genre,book_text
0,অভীক সরকার,male,খোঁড়া ভৈরবীর মাঠ,horror,খোঁড়া ভৈরবীর মাঠ অভীক সরকার roofreadin...
1,তারাদাস বন্দ্যোপাধ্যায়,male,অলাতচক্র,horror,অলাতচক্র তারাদাস বন্দ্যোপাধ্যায় yPED By ...
2,তারাদাস বন্দ্যোপাধ্যায়,male,তারানাথ তান্ত্রিক,horror,এই গ্রন্থের অন্তর্ভুক্ত কাহিনী আমার চিন্তাপ্রস...
3,শরদিন্দু বন্দ্যোপাধ্যায়,male,সত্যান্বেষী,detective,সত্যান্বেষী শরদিন্দু বন্দ্যোপাধ্যায় ...
4,শরদিন্দু বন্দ্যোপাধ্যায়,male,পথের কাঁটা,detective,পথের কাঁটা শরদিন্দু বন্দ্যোপাধ্যায় Boo...


Merge All JSON  

In [ ]:
# Read the JSON file
df1 = pd.read_json('/content/drive/MyDrive/Author Gender Prediction/amir.json')
df2 = pd.read_json('/content/drive/MyDrive/Author Gender Prediction/robin.json')


# Normalize the specific column if it contains nested JSON data
normalized_df1 = pd.json_normalize(df1["books"])
normalized_df2 = pd.json_normalize(df2["books"])

data = [normalized_df1, normalized_df2]

combined_df = pd.concat(data)

combined_df.shape

# Convert the combined DataFrame to CSV
combined_df.to_csv('/content/drive/MyDrive/Author Gender Prediction/dataset.csv', index=False)

Merge All CSV

In [ ]:
import pandas as pd

# Load the two CSV files
file1 = '/content/drive/MyDrive/Author Gender Prediction/book_data (1).csv'
file2 = '/content/drive/MyDrive/Author Gender Prediction/dataset.csv'

# Read both CSV files into DataFrames
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Merge the DataFrames (you can specify how to merge, e.g., append or merge on a column)
merged_df = pd.concat([df1, df2], ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv', index=False)

print("Files merged and saved as newdataset.csv")


Remove English Charecters

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv')
def remove_english(text):
	out = re.compile('[a-zA-Z0-9]+', flags=re.I) #remove english
	text=out.sub(r'', text)

	return text

df["cleaned_text"] = df['book_text'].apply(lambda x: remove_english(str(x)))

csv_file = '/content/drive/MyDrive/Author Gender Prediction/newdataset.csv'

# Read the existing CSV file into a pandas DataFrame
existing_data = pd.read_csv(csv_file)

# Generate the new column as a pandas Series or list
new_column_data = df["cleaned_text"]  # Replace with your actual new column data

# Add the new column to the existing DataFrame
existing_data['remove_english'] = new_column_data

# Write the modified DataFrame back to the CSV file
existing_data.to_csv(csv_file, index=False)

Remove Duplicate

In [ ]:
import pandas as pd

# Load the newdataset.csv
file_path = '/content/drive/MyDrive/Author Gender Prediction/newdataset.csv'
df = pd.read_csv(file_path)

# Check for duplicates
duplicates = df[df.duplicated()]

# Print the duplicate rows
if not duplicates.empty:
    print("Duplicate rows found:")
    print(duplicates)
else:
    print("No duplicate rows found.")

No duplicate rows found.


In [ ]:
# Remove duplicates and keep only the first occurrence
df_no_duplicates = df.drop_duplicates()

# Save the DataFrame without duplicates to a new CSV file
df_no_duplicates.to_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv', index=False)

print("Duplicates removed, and file saved as newdataset_no_duplicates.csv")

Duplicates removed, and file saved as newdataset_no_duplicates.csv


In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/dataset.csv')

# Replace "কি" to "কী" in the "book_text" column
df['remove_english'] = df['remove_english'].str.replace('কি', 'কী')

# Write the updated CSV file
df.to_csv('/content/drive/MyDrive/Author Gender Prediction/dataset.csv', index=False)

Converted Pronoun's Shadu to Cholit

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv')

# Initialize a dictionary with Sadhu to Cholit mappings
replacements = {
    'তুই': 'তুমি',
    'তিনি': 'সে',
    'ইহা': 'এ',
    'উহা': 'ও',
    'তাহার': 'তার',
    'তোর': 'আপনার',
    'কি': 'কী',
    'সমস্ত': 'সব',
    'সর্ব': 'সব',
    'অনেকেই': 'অনেকে',
    'যিনি': 'যে',
    'যাহা': 'যা'
}

# Column where conversion is needed
book_text_column = 'remove_english'

# Initialize a counter for the number of replacements made
conversion_count = {key: 0 for key in replacements.keys()}

# Function to apply the replacement and count conversions
def convert_sadhu_to_cholit(text):
    for sadhu, cholit in replacements.items():
        # Count occurrences of the sadhu word in the text
        conversion_count[sadhu] += text.count(sadhu)
        # Replace sadhu with cholit
        text = text.replace(sadhu, cholit)
    return text

# Apply the conversion function to the 'book_text' column
df[book_text_column] = df[book_text_column].apply(convert_sadhu_to_cholit)

# Save the modified dataframe back to a CSV file if needed
df.to_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv', index=False)

# Output the number of conversions for each sadhu word
for sadhu, count in conversion_count.items():
    print(f"Converted '{sadhu}' to '{replacements[sadhu]}' {count} times.")

print("Conversion completed!")


Converted 'তুই' to 'তুমি' 6624 times.
Converted 'তিনি' to 'সে' 29240 times.
Converted 'ইহা' to 'এ' 3305 times.
Converted 'উহা' to 'ও' 445 times.
Converted 'তাহার' to 'তার' 12339 times.
Converted 'তোর' to 'আপনার' 7631 times.
Converted 'কি' to 'কী' 200351 times.
Converted 'সমস্ত' to 'সব' 4418 times.
Converted 'সর্ব' to 'সব' 4231 times.
Converted 'অনেকেই' to 'অনেকে' 1021 times.
Converted 'যিনি' to 'যে' 988 times.
Converted 'যাহা' to 'যা' 1792 times.
Conversion completed!


Chunking Dataset

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv')

# Function to split text into sentences
def split_into_sentences(text):
    return sent_tokenize(text)

# Function to clean and split text
def clean_and_split_text(text, min_length=4000, split_length=4000):
    sentences = split_into_sentences(text)

    # If the text is already short, return it as is
    if len(text.split()) <= min_length:
        return [text]

    # Remove the first and last pages (assuming pages are roughly 300 words)
    words = ' '.join(text.split()[300:-300])
    cleaned_sentences = split_into_sentences(words)

    chunks = []
    current_chunk = ""

    for sentence in cleaned_sentences:
        if len(current_chunk.split()) + len(sentence.split()) <= split_length:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# Apply the cleaning and splitting function to the 'remove_english' column
df['text'] = df['remove_english'].apply(clean_and_split_text)

# Explode the list of chunks into separate rows
df = df.explode('text').reset_index(drop=True)

# Save the cleaned and split dataset to a new CSV file
df.to_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv', index=False)

print("Data splitting completed.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Data cleaning and splitting completed. The cleaned dataset has been saved to 'cleaned_and_split_dataset.csv'.


Remove Unnessesary Column

In [ ]:
input_file = '/content/drive/MyDrive/Author Gender Prediction/newdataset.csv'
output_file = '/content/drive/MyDrive/Author Gender Prediction/newdataset.csv'
columns_to_remove = ['book_text','remove_english']  # List the column names you want to remove

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(input_file)

# Remove the specified columns from the DataFrame
data = data.drop(columns=columns_to_remove)

# Write the modified DataFrame to a new CSV file
data.to_csv(output_file, index=False)

# # Specify the column name and value to match for deletion
# column_name = 'author'
# value_to_match = 'dummy'

# # Delete rows based on the specified criteria
# data = data[data[column_name] != value_to_match]

# Save the modified DataFrame to a new file or overwrite the existing one
data.to_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv', index=False)

In [ ]:
import csv
import sys

# Increase the field size limit to handle large CSV fields
csv.field_size_limit(sys.maxsize)

input_file = '/content/drive/MyDrive/Author Gender Prediction/newdataset.csv'
output_file = '/content/drive/MyDrive/Author Gender Prediction/reduced_dataset.csv'
columns_to_remove = ['book_text', 'remove_english']  # Columns to be removed

# Open the input CSV file for reading
with open(input_file, mode='r', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)

    # Get all fieldnames except the ones to remove
    fieldnames = [field for field in reader.fieldnames if field not in columns_to_remove]

    # Open the output CSV file for writing
    with open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

        # Write the new header (without removed columns)
        writer.writeheader()

        # Write each row, excluding the removed columns
        for row in reader:
            filtered_row = {field: row[field] for field in fieldnames}
            writer.writerow(filtered_row)

# Optionally, overwrite the original file with the reduced file
!mv /content/drive/MyDrive/Author\ Gender\ Prediction/reduced_dataset.csv /content/drive/MyDrive/Author\ Gender\ Prediction/newdataset.csv


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv')
# Change the maximum number of rows and columns displayed
pd.set_option('display.max_rows', None)  # Set to None to display all rows
pd.set_option('display.max_columns', None)  # Set to None to display all columns

# Now, when you print your DataFrame, it will display all rows and columns
df


In [ ]:
import pandas as pd

# Path to your original CSV file in Google Drive
csv_path = '/content/drive/MyDrive/Author Gender Prediction/dataset.csv'  # Update with your actual file path

# Read the CSV file
df = pd.read_csv(csv_path)

# Select only the desired columns
filtered_df = df[['title', 'author', 'genre', 'gender']]

# Path to save the new Excel file in Google Drive
excel_path = '/content/drive/MyDrive/BOOK/filtered_data.xlsx'  # Update with your folder path

# Save the filtered data to a new Excel file
filtered_df.to_excel(excel_path, index=False, engine='openpyxl')

print("New Excel file created successfully!")


New Excel file created successfully!


Create 4000 Word Segment

In [ ]:
#4000 Word Segment
word_length = 4000
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv')

def get_middle_max_4000_word(input_string):
    # Split the input string into a list of sentences
    sentences = re.split(r'(?<=[।!?])\s+', input_string)

    # Initialize variables
    middle_index = len(sentences) // 2
    num_words = 0
    extracted_sentences = []

    # Find the middle maximum 4000 words containing complete sentences
    while num_words < word_length and middle_index >= 0 and middle_index < len(sentences):
        words = sentences[middle_index].split()
        if num_words + len(words) <= word_length:
            num_words += len(words)
            extracted_sentences.append(sentences[middle_index])
        middle_index -= 1

    # Join the extracted sentences into a string
    output_string = ' '.join(extracted_sentences)

    return output_string

df["cleaned_text"] = df['text'].apply(lambda x: get_middle_max_4000_word(str(x)))

csv_file = '/content/drive/MyDrive/Author Gender Prediction/newdataset.csv'

# Read the existing CSV file into a pandas DataFrame
existing_data = pd.read_csv(csv_file)

# Generate the new column as a pandas Series or list
new_column_data = df["cleaned_text"]  # Replace with your actual new column data

# Add the new column to the existing DataFrame
existing_data['pre_processed_data_1'] = new_column_data

# Write the modified DataFrame back to the CSV file
existing_data.to_csv(csv_file, index=False)

Create 3000 Word Segment

In [ ]:
#3000 Word Segment
word_length = 3000
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv')

def get_middle_max_4000_word(input_string):
    # Split the input string into a list of sentences
    sentences = re.split(r'(?<=[।!?])\s+', input_string)

    # Initialize variables
    middle_index = len(sentences) // 2
    num_words = 0
    extracted_sentences = []

    # Find the middle maximum 4000 words containing complete sentences
    while num_words < word_length and middle_index >= 0 and middle_index < len(sentences):
        words = sentences[middle_index].split()
        if num_words + len(words) <= word_length:
            num_words += len(words)
            extracted_sentences.append(sentences[middle_index])
        middle_index -= 1

    # Join the extracted sentences into a string
    output_string = ' '.join(extracted_sentences)

    return output_string

df["cleaned_text"] = df['pre_processed_data_1'].apply(lambda x: get_middle_max_4000_word(str(x)))

csv_file = '/content/drive/MyDrive/Author Gender Prediction/newdataset.csv'

# Read the existing CSV file into a pandas DataFrame
existing_data = pd.read_csv(csv_file)

# Generate the new column as a pandas Series or list
new_column_data = df["cleaned_text"]  # Replace with your actual new column data

# Add the new column to the existing DataFrame
existing_data['pre_processed_data_2'] = new_column_data

# Write the modified DataFrame back to the CSV file
existing_data.to_csv(csv_file, index=False)

Create 2000 Word Segment

In [ ]:
#2000 Word Segment
word_length = 2000
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv')

def get_middle_max_4000_word(input_string):
    # Split the input string into a list of sentences
    sentences = re.split(r'(?<=[।!?])\s+', input_string)

    # Initialize variables
    middle_index = len(sentences) // 2
    num_words = 0
    extracted_sentences = []

    # Find the middle maximum 4000 words containing complete sentences
    while num_words < word_length and middle_index >= 0 and middle_index < len(sentences):
        words = sentences[middle_index].split()
        if num_words + len(words) <= word_length:
            num_words += len(words)
            extracted_sentences.append(sentences[middle_index])
        middle_index -= 1

    # Join the extracted sentences into a string
    output_string = ' '.join(extracted_sentences)

    return output_string

df["cleaned_text"] = df['pre_processed_data_2'].apply(lambda x: get_middle_max_4000_word(str(x)))

csv_file = '/content/drive/MyDrive/Author Gender Prediction/newdataset.csv'

# Read the existing CSV file into a pandas DataFrame
existing_data = pd.read_csv(csv_file)

# Generate the new column as a pandas Series or list
new_column_data = df["cleaned_text"]  # Replace with your actual new column data

# Add the new column to the existing DataFrame
existing_data['pre_processed_data_3'] = new_column_data

# Write the modified DataFrame back to the CSV file
existing_data.to_csv(csv_file, index=False)

Remove Unnessesary Column

In [ ]:
input_file = '/content/drive/MyDrive/Author Gender Prediction/newdataset.csv'
output_file = '/content/drive/MyDrive/Author Gender Prediction/newdataset.csv'
columns_to_remove = ['text']  # List the column names you want to remove

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(input_file)

# Remove the specified columns from the DataFrame
data = data.drop(columns=columns_to_remove)

# Write the modified DataFrame to a new CSV file
data.to_csv(output_file, index=False)

# # Specify the column name and value to match for deletion
# column_name = 'author'
# value_to_match = 'dummy'

# # Delete rows based on the specified criteria
# data = data[data[column_name] != value_to_match]

# Save the modified DataFrame to a new file or overwrite the existing one
data.to_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset_1.csv')
# Change the maximum number of rows and columns displayed
pd.set_option('display.max_rows', None)  # Set to None to display all rows
pd.set_option('display.max_columns', None)  # Set to None to display all columns

# Now, when you print your DataFrame, it will display all rows and columns
df


,author,gender,title,genre,text
0,অভীক সরকার,male,খোঁড়া ভৈরবীর মাঠ,horror,বানানাে। কামাখ্যা থেকে বিখ্যাত অঘােরী সাধক কাল...
1,অভীক সরকার,male,খোঁড়া ভৈরবীর মাঠ,horror,থরথর করে কাঁপতে থাকা আলােয় রাখহরি দেখে প্রকাণ...
2,তারাদাস বন্দ্যোপাধ্যায়,male,অলাতচক্র,horror,জমে উঠেছে দারুণ। বেশ বড় বাড়ি তারপরে দুর্গামণ...
3,তারাদাস বন্দ্যোপাধ্যায়,male,অলাতচক্র,horror,এ গ্রামে দেখেছি বলে তো মনে হল না— বাবা প্রায় ...
4,তারাদাস বন্দ্যোপাধ্যায়,male,অলাতচক্র,horror,—না তাই জিজ্ঞেস করছি— —আমি এই গ্রামেরই ভেতরে থ...
5,তারাদাস বন্দ্যোপাধ্যায়,male,অলাতচক্র,horror,না তা করা যায় না। এইখানেই সাধকের পরীক্ষা। ক্ষ...
6,তারাদাস বন্দ্যোপাধ্যায়,male,অলাতচক্র,horror,উত্তেজিত হওয়া তর্কে পরাজয়ের লক্ষণ । —বেশ তো ...
7,তারাদাস বন্দ্যোপাধ্যায়,male,অলাতচক্র,horror,তখন ব্রাহ্মমুহুর্ত। পূব আকাশে প্রাক্ উষার হালক...
8,তারাদাস বন্দ্যোপাধ্যায়,male,অলাতচক্র,horror,পেছনে সব আসছে বোধহয়— দেবদর্শনের স্ত্রী কীন্তু...
9,তারাদাস বন্দ্যোপাধ্যায়,male,অলাতচক্র,horror,অফিসের নির্দেশ জানিয়ে বাবাকে চিঠি লিখলাম আশা ...


OverSample Dataset

In [ ]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset.csv')

# Display the initial class distribution
print("Initial class distribution:")
print(df['gender'].value_counts())

# Separate features and target
X = df[['author', 'title', 'genre', 'pre_processed_data_1', 'pre_processed_data_2', 'pre_processed_data_3']]
y = df['gender']

# Initialize the RandomOverSampler
ros = RandomOverSampler(sampling_strategy='minority')

# Oversample the minority class
X_res, y_res = ros.fit_resample(X, y)

# Convert back to DataFrame
df_resampled = pd.DataFrame(X_res, columns=['author', 'title', 'genre', 'pre_processed_data_1', 'pre_processed_data_2', 'pre_processed_data_3'])
df_resampled['gender'] = y_res

# Display the new class distribution
print("New class distribution:")
print(df_resampled['gender'].value_counts())

# Save the balanced dataset to a new CSV file
df_resampled.to_csv('/content/drive/MyDrive/Author Gender Prediction/balanced_dataset.csv', index=False)

print("The balanced dataset has been saved to 'balanced_dataset.csv'.")


Initial class distribution:
gender
male      2156
female     278
Name: count, dtype: int64
New class distribution:
gender
male      2156
female    2156
Name: count, dtype: int64
The balanced dataset has been saved to 'balanced_dataset.csv'.


Frequency Count PRONOUNS

In [ ]:
# Set a larger field size limit
csv.field_size_limit(sys.maxsize)

csv_file = '/content/drive/MyDrive/Author Gender Prediction/balanced_dataset.csv'

with open(csv_file, mode='r', encoding='utf-8', newline='') as file:
    reader = csv.DictReader(file)
    rows = list(reader)


# Specify the path to the CSV file
csv_file = "/content/drive/MyDrive/Author Gender Prediction/balanced_dataset.csv"

# Specify the column name to count the occurrences in
column_name = "pre_processed_data_1"

# Specify the Bangla words to count
words = ['আমি', 'তুমি', 'আপনি', 'সে', 'এ','ও','আমার','আপনার','তার','সেই','কার','কেউ','কোন','কত', 'কী','সব','অনেকে','একজন', 'যে','যা']

# Read the CSV file and access the values from the specified column
with open(csv_file, mode='r', encoding='utf-8', newline='') as file:
    reader = csv.DictReader(file)
    rows = list(reader)

# Add separate columns for each word and initialize their values to 0
for word in words:
    new_column_name = f"{word}"
    for row in rows:
        row[new_column_name] = 0

# Iterate over the words and count their occurrences in each row
for word in words:
    for row in rows:
        count = row[column_name].count(word)
        row[f"{word}"] = count

# Write the updated rows with the new columns to a new CSV file
new_csv_file = "/content/drive/MyDrive/Author Gender Prediction/balanced_dataset.csv"
fieldnames = list(rows[0].keys())
with open(new_csv_file, mode='w', encoding='utf-8', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/balanced_dataset.csv')
# Change the maximum number of rows and columns displayed
pd.set_option('display.max_rows', None)  # Set to None to display all rows
pd.set_option('display.max_columns', None)  # Set to None to display all columns

# Now, when you print your DataFrame, it will display all rows and columns
df


In [ ]:
import pandas as pd

# Load your CSV file into a pandas DataFrame
data = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset_3_updated.csv')

# Calculate total occurrences of each pronoun
pronouns = ['আমি', 'তুমি', 'আপনি', 'সে', 'এ','ও','আমার','আপনার','তার','সেই','কার','কেউ','কোন','কত', 'কী','সব','অনেকে','একজন', 'যে','যা']  # Add all pronouns

# Calculate total occurrences of each pronoun
total_occurrences = data[pronouns].sum()

# Create a DataFrame for the pronoun occurrences table
pronoun_table = pd.DataFrame({
    'Pronoun': pronouns,
    'Total Occurrences': total_occurrences
})

# Sort the table by occurrences in descending order
pronoun_table = pronoun_table.sort_values(by='Total Occurrences', ascending=False)

# Print the pronoun occurrences table
pronoun_table



,Pronoun,Total Occurrences
এ,এ,790724
ও,ও,528936
সে,সে,418457
কী,কী,401243
যা,যা,375573
যে,যে,250962
তার,তার,201091
কার,কার,130990
আমি,আমি,128588
আমার,আমার,121567


In [ ]:
import pandas as pd

# Load CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset_3_updated.csv')  # Replace 'your_file.csv' with your file name

# Calculate average pronoun usage by gender
pronouns = ['আমি', 'তুমি', 'আপনি', 'সে', 'এ','ও','আমার','আপনার','তার','সেই','কার','কেউ','কোন','কত', 'কী','সব','অনেকে','একজন', 'যে','যা']
average_usage = []

for pronoun in pronouns:
    male_avg = df[df['gender'] == 'male'][pronoun].mean()
    female_avg = df[df['gender'] == 'female'][pronoun].mean()
    average_usage.append({'Pronoun': pronoun, 'Male': male_avg, 'Female': female_avg})

average_df = pd.DataFrame(average_usage)
average_df.sort_values(by='Male', ascending=True)
average_df

,Pronoun,Male,Female
0,আমি,15.172545,15.472831
1,তুমি,6.321973,5.107960
2,আপনি,4.212107,1.329600
3,সে,52.376072,47.351525
4,এ,99.432793,89.014299
5,ও,58.764299,67.292898
6,আমার,13.205910,15.766206
7,আপনার,5.414681,2.875357
8,তার,25.302908,22.621544
9,সেই,6.240229,6.541706


Support Vector Machine

In [ ]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset_3_updated.csv')

# Define the pronouns
pronouns = ['আমি', 'তুমি', 'আপনি', 'সে', 'এ', 'ও','আমার','আপনার','তার','সেই','কার','কেউ','কোন','কত',
            'কী','সব','অনেকে','একজন', 'যে','যা']

# Calculate relative pronoun frequencies
for pronoun in pronouns:
    data[f'rel_{pronoun}'] = data[pronoun] / 2000  # 'total_words' is the total word count in the segment

# Define features and target
X = data[[f'rel_{pronoun}' for pronoun in pronouns]]
y = data['gender']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define hyperparameter grid
param_grid = {
    'C': [0.1, 1.0, 10.0],
    'gamma': ['scale', 'auto']
}

# Create the SVM classifier
svm = SVC(random_state=42)

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the model with the best hyperparameters
best_svm = SVC(random_state=42, **best_params)
best_svm.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = best_svm.predict(X_test)

# Perform 5-fold cross-validation
cross_val_scores = cross_val_score(best_svm, X, y, cv=5)

# Print the performance metrics for each fold
print("Cross-validation scores:", cross_val_scores)

# Calculate the average performance across all folds
avg_score = cross_val_scores.mean()
print("Average score:", avg_score)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='male')
recall = recall_score(y_test, y_pred, pos_label='male')
f1 = f1_score(y_test, y_pred, pos_label='male')
mcc = matthews_corrcoef(y_test, y_pred)
confusion_svm = confusion_matrix(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("MCC:", mcc)
print(classification_report(y_test, y_pred))
print("Confusion Matrix for SVM:")
print(confusion_svm)


Best Hyperparameters: {'C': 10.0, 'gamma': 'scale'}
Cross-validation scores: [0.81119714 0.78916021 0.77234803 0.79976162 0.82240763]
Average score: 0.7989749276095865
Accuracy: 0.8409767718880286
Precision: 0.8638743455497382
Recall: 0.8019441069258809
F1 Score: 0.831758034026465
MCC: 0.6830854161511174
              precision    recall  f1-score   support

      female       0.82      0.88      0.85       856
        male       0.86      0.80      0.83       823

    accuracy                           0.84      1679
   macro avg       0.84      0.84      0.84      1679
weighted avg       0.84      0.84      0.84      1679

Confusion Matrix for SVM:
[[752 104]
 [163 660]]


In [ ]:
X.to_csv('/content/drive/MyDrive/Author Gender Prediction/filtered_bangla.csv', index=False)

Naive Bayes

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset_3_updated.csv')

# Define the pronouns
pronouns = ['আমি', 'তুমি', 'আপনি', 'সে', 'এ', 'ও','আমার','আপনার','তার','সেই','কার','কেউ','কোন','কত',
            'কী','সব','অনেকে','একজন', 'যে','যা']
# Calculate relative pronoun frequencies
for pronoun in pronouns:
    data[f'rel_{pronoun}'] = data[pronoun] / 2000  # 'total_words' is the total word count in the segment

# Define features and target
X = data[[f'rel_{pronoun}' for pronoun in pronouns]]
y = data['gender']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
param_grid = {}

# Create the Naive Bayes classifier
nb = GaussianNB()

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(nb, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model
best_nb = grid_search.best_estimator_

# Train the model with the best hyperparameters
best_nb.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = best_nb.predict(X_test)

# Perform 5-fold cross-validation
cross_val_scores = cross_val_score(best_nb,  X, y, cv=5)

# Print the performance metrics for each fold
print("Cross-validation scores:", cross_val_scores)

# Calculate the average performance across all folds
avg_score = cross_val_scores.mean()
print("Average score:", avg_score)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='male')
recall = recall_score(y_test, y_pred, pos_label='male')
f1 = f1_score(y_test, y_pred, pos_label='male')
mcc = matthews_corrcoef(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("MCC:", mcc)
print(classification_report(y_test, y_pred))
confusion_nb = confusion_matrix(y_test, y_pred)
print("Confusion Matrix for NB:")
print(confusion_nb)


Cross-validation scores: [0.77367481 0.65217391 0.65375447 0.65494636 0.66924911]
Average score: 0.6807597319762246
Accuracy: 0.6956521739130435
Precision: 0.6959798994974874
Recall: 0.6731470230862697
F1 Score: 0.6843730697961705
MCC: 0.390886431334663
              precision    recall  f1-score   support

      female       0.70      0.72      0.71       856
        male       0.70      0.67      0.68       823

    accuracy                           0.70      1679
   macro avg       0.70      0.70      0.70      1679
weighted avg       0.70      0.70      0.70      1679

Confusion Matrix for NB:
[[614 242]
 [269 554]]


Decision Tree

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset_3_updated.csv')

# Define the pronouns
pronouns = ['আমি', 'তুমি', 'আপনি', 'সে', 'এ', 'ও','আমার','আপনার','তার','সেই','কার','কেউ','কোন','কত',
            'কী','সব','অনেকে','একজন', 'যে','যা']

# Calculate relative pronoun frequencies
for pronoun in pronouns:
    data[f'rel_{pronoun}'] = data[pronoun] / 2000  # 'total_words' is the total word count in the segment

# Define features and target
X = data[[f'rel_{pronoun}' for pronoun in pronouns]]
y = data['gender']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set random seeds for reproducibility
np.random.seed(0)

# Initialize the decision tree model
model = DecisionTreeClassifier()

# Define the hyperparameter grid
param_grid = {
    'max_depth': [6, 7, 8],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 3],
    'ccp_alpha': [0.001, 0.005],
    'criterion': ['gini', 'entropy']  # Add criterion to grid search
}



# Perform a grid search with a fixed random seed
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model and predict on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Set random seed for cross-validation
np.random.seed(0)

# Perform 5-fold cross-validation
cross_val_scores = cross_val_score(best_model, X, y, cv=5)

# Calculate the average performance across all folds
avg_score = cross_val_scores.mean()
print("Cross-validation scores:", cross_val_scores)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='male')
recall = recall_score(y_test, y_pred, pos_label='male')
f1 = f1_score(y_test, y_pred, pos_label='male')
mcc = matthews_corrcoef(y_test, y_pred)

# Print the results
print("Best Hyperparameters:", grid_search.best_params_)
print("Average score:", avg_score)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("MCC:", mcc)
print(classification_report(y_test, y_pred))

print("Confusion Matrix for NB:")
print(confusion_matrix(y_test, y_pred))

Cross-validation scores: [0.83561644 0.84157236 0.81585221 0.80274136 0.83969011]
Best Hyperparameters: {'ccp_alpha': 0.001, 'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 3, 'min_samples_split': 5}
Average score: 0.8270944947791585
Accuracy: 0.8564621798689697
Precision: 0.907563025210084
Recall: 0.787363304981774
F1 Score: 0.8432010409889394
MCC: 0.7181938814510078
              precision    recall  f1-score   support

      female       0.82      0.92      0.87       856
        male       0.91      0.79      0.84       823

    accuracy                           0.86      1679
   macro avg       0.86      0.86      0.86      1679
weighted avg       0.86      0.86      0.86      1679

Confusion Matrix for NB:
[[790  66]
 [175 648]]


Random Foret

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/balanced_dataset.csv')

# Define the pronouns
pronouns = ['আমি', 'তুমি', 'আপনি', 'সে', 'এ', 'ও','আমার','আপনার','তার','সেই','কার','কেউ','কোন','কত',
            'কী','সব','অনেকে','একজন', 'যে','যা']
# Calculate relative pronoun frequencies
for pronoun in pronouns:
    data[f'rel_{pronoun}'] = data[pronoun] / 4000  # 'total_words' is the total word count in the segment

# Define features and target
X = data[[f'rel_{pronoun}' for pronoun in pronouns]]
y = data['gender']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
rf = RandomForestClassifier(random_state=0)

param_grid = {
    'n_estimators': [100, 200, 300],      # Number of trees in the forest
    'max_depth': [None, 5, 10, 20],       # Maximum depth of the trees
    'min_samples_split': [2, 5, 10]       # Minimum number of samples required to split an internal node
}

grid_search = GridSearchCV(rf, param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_rf = RandomForestClassifier(random_state=42, **best_params)
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)

# Perform 5-fold cross-validation
cross_val_scores = cross_val_score(best_rf,  X, y, cv=5)

# Print the performance metrics for each fold
print("Cross-validation scores:", cross_val_scores)

# Calculate the average performance across all folds
avg_score = cross_val_scores.mean()
print("Average score:", avg_score)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='male')
recall = recall_score(y_test, y_pred, pos_label='male')
f1 = f1_score(y_test, y_pred, pos_label='male')
mcc = matthews_corrcoef(y_test, y_pred)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("MCC:", mcc)
print(classification_report(y_test, y_pred))
print("Confusion Matrix for RF:")
print(confusion_matrix(y_test, y_pred))


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, classification_report, confusion_matrix

# Load data
data = pd.read_csv('/content/drive/MyDrive/Author Gender Prediction/newdataset_3_updated.csv')

# Define the pronouns
pronouns = ['আমি', 'তুমি', 'আপনি', 'সে', 'এ', 'ও','আমার','আপনার','তার','সেই','কার','কেউ','কোন','কত',
            'কী','সব','অনেকে','একজন', 'যে','যা']

# Calculate relative pronoun frequencies
for pronoun in pronouns:
    data[f'rel_{pronoun}'] = data[pronoun] / 2000  # assuming 4000 is the total word count in the segment

# Define features and target
X = data[[f'rel_{pronoun}' for pronoun in pronouns]]
y = data['gender']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 75, 100],  # Try adding 100
    'max_depth': [3, 4, 5, 6],  # Include 6
    'min_samples_split': [5, 10, 15],  # Include 15
    'min_samples_leaf': [2, 3, 4],  # Add 4
    'max_features': ['sqrt', 'log2'],  # Keep these options
    'bootstrap': [True],
    'criterion': ['gini', 'entropy']  # Test both criteria
}





# Perform grid search
grid_search = GridSearchCV(rf, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best parameters and train the best model
best_params = grid_search.best_params_
best_rf = RandomForestClassifier(random_state=42, **best_params)
best_rf.fit(X_train, y_train)

# Make predictions
y_pred = best_rf.predict(X_test)

# Perform 5-fold cross-validation
cross_val_scores = cross_val_score(best_rf, X, y, cv=5)

# Print the performance metrics for each fold
print("Cross-validation scores:", cross_val_scores)

# Calculate the average performance across all folds
avg_score = cross_val_scores.mean()
print("Average score:", avg_score)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='male')
recall = recall_score(y_test, y_pred, pos_label='male')
f1 = f1_score(y_test, y_pred, pos_label='male')
mcc = matthews_corrcoef(y_test, y_pred)

# Print the results
print("Best Hyperparameters:", grid_search.best_params_)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("MCC:", mcc)
print(classification_report(y_test, y_pred))
print("Confusion Matrix for RF:")
print(confusion_matrix(y_test, y_pred))

# Extract feature importance
feature_importance = best_rf.feature_importances_
pronouns = [f'rel_{pronoun}' for pronoun in pronouns]

# Create a DataFrame for feature importance
importance_df = pd.DataFrame({'Pronoun': pronouns, 'Feature Importance': feature_importance})

# Sort by importance
importance_df = importance_df.sort_values(by='Feature Importance', ascending=False)

# Save to CSV (optional, for easy loading into Excel)
importance_df.to_csv('feature_importance.csv', index=False)

print(importance_df)


Cross-validation scores: [0.86956522 0.82549136 0.80512515 0.80631704 0.84982122]
Average score: 0.8312639980236831
Best Hyperparameters: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 15, 'n_estimators': 75}
Accuracy: 0.8552709946396665
Precision: 0.8856382978723404
Recall: 0.8092345078979344
F1 Score: 0.8457142857142858
MCC: 0.71251110082339
              precision    recall  f1-score   support

      female       0.83      0.90      0.86       856
        male       0.89      0.81      0.85       823

    accuracy                           0.86      1679
   macro avg       0.86      0.85      0.85      1679
weighted avg       0.86      0.86      0.85      1679

Confusion Matrix for RF:
[[770  86]
 [157 666]]
      Pronoun  Feature Importance
2    rel_আপনি            0.134598
7   rel_আপনার            0.118796
5       rel_ও            0.092407
4       rel_এ            0.074024
17   rel_একজন            0.0707

In [ ]:
df = pd.read_csv('/content/feature_importance.csv')
# Change the maximum number of rows and columns displayed
pd.set_option('display.max_rows', None)  # Set to None to display all rows
pd.set_option('display.max_columns', None)  # Set to None to display all columns

# Now, when you print your DataFrame, it will display all rows and columns
df

,Pronoun,Feature Importance
0,rel_আপনি,0.134598
1,rel_আপনার,0.118796
2,rel_ও,0.092407
3,rel_এ,0.074024
4,rel_একজন,0.070712
5,rel_আমার,0.070669
6,rel_কেউ,0.053609
7,rel_আমি,0.053211
8,rel_তার,0.048813
9,rel_কার,0.046321
